
### Time travel

* we can query older version of data
* if you deleted data from table by accidentily you can get back that data by using below method







In [0]:
%sql 
describe history new_catalog.default.test

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2025-12-10T09:31:42.000Z,72252168792421,afzalkhan098368@gmail.com,UPDATE,Map(predicate -> []),null,List(3285367649457626),1210-090635-krt3iqan-v2n,3,WriteSerializable,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 2601, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1241, numDeletionVectorsUpdated -> 0, scanTimeMs -> 18, numAddedFiles -> 1, numUpdatedRows -> 6, numAddedBytes -> 1655, rewriteTimeMs -> 1205)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
3,2025-12-10T09:11:15.000Z,72252168792421,afzalkhan098368@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> true, partitionBy -> [])",null,List(3285367649457626),1210-090635-krt3iqan-v2n,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutputBytes -> 868)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
2,2025-12-10T09:11:13.000Z,72252168792421,afzalkhan098368@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3285367649457626),1210-090635-krt3iqan-v2n,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutputBytes -> 862)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
1,2025-12-10T09:11:11.000Z,72252168792421,afzalkhan098368@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> true, partitionBy -> [])",null,List(3285367649457626),1210-090635-krt3iqan-v2n,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutputBytes -> 871)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
0,2025-12-10T09:08:08.000Z,72252168792421,afzalkhan098368@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true"",""delta.writePartitionColumnsToParquet"":""true"",""delta.enableRowTracking"":""true"",""delta.rowTracking.materializedRowCommitVersionColumnName"":""_row-commit-version-col-7d220760-b039-4fe5-9942-452ff0b0d943"",""delta.rowTracking.materializedRowIdColumnName"":""_row-id-col-593b3964-83d4-4aa6-9e95-49799d925435""}, statsOnLoad -> false)",null,List(3285367649457626),1210-090635-krt3iqan-v2n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


###Method 1 - by using timestamp

* you can restore table by timestamp method

Restore table new_catalog.default.test timestamp as of '2025-12-10T09:11:15.000+00:00'


In [0]:
%sql
select * from new_catalog.default.test timestamp as of '2025-12-10T09:11:15.000+00:00'

sno,sname
1,sunny
2,Rockey
5,surya
6,sandy
3,Raj
4,danny



###Method 2 - by using @v

* You can restore table by using @v

Restore table new_catalog.default.test@v2

In [0]:
%sql
select * from new_catalog.default.test@v2

sno,sname
1,sunny
2,Rockey
3,Raj
4,danny



###Method 3 - by using version of

* You can restore table by using version of 

Restore table new_catalog.default.test version as of 3

In [0]:
%sql
select * from new_catalog.default.test version as of 3

sno,sname
1,sunny
2,Rockey
5,surya
6,sandy
3,Raj
4,danny



### OPTIMIZE command

* the OPTIMIZE command is used to compact small files into large file and improve query performance for Delta tables. It can also include Z-Ordering to optimize data skipping.

###Syntax -

Optimize table Zorder by column_name


In [0]:
%sql

optimize new_catalog.default.test zorder by sno;

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1655), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1765445056271, 1765445057034, 8, 0, null, List(0, 0), null, 2, 2, 0, 0, null)"



###Auto Optimize in Databricks

Databricks also offers automatic optimization:

In [0]:
%sql
ALTER TABLE new_catalog.default.test SET TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = true,
  'delta.autoOptimize.autoCompact' = true
);



### VACUUM command

* the VACUUM command is used to remove old files that are no longer needed for the Delta table’s state. This helps reduce storage costs and improve performance.

###Syntax

Vaccum table_name RETAIN NUMBER HOURS

In [0]:
%sql

 VACUUM new_catalog.default.test RETAIN 168 HOURS;  -- 7 days



###Common values:

* 168 HOURS — 7 days (default)

* 720 HOURS — 30 days

* 0 HOURS — remove everything not needed for the latest snapshot


###Disable Retention Check 

In [0]:
%sql

SET spark.databricks.delta.retentionDurationCheck.enabled = false;



📌 Notes & Best Practices

* Don’t set retention < 7 days unless absolutely necessary (risk of breaking time travel).

* Always run DRY RUN before aggressive vacuuming.

* VACUUM doesn't rewrite or reorganize data—only deletes old files.

* Use OPTIMIZE + ZORDER before VACUUM for best performance.